In [1]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
import os



# Load MNIST Dataset

In [2]:
# Make torch deterministic
_ = torch.manual_seed(0)

In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307, ), (0.3081, ))])

# Load the MNIST dataset
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# Create a dataloader for the training
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=10, shuffle=True)

# Load the MNIST test set
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=10, shuffle=True)

# Define the device 
device = "cpu"


# Define the model

In [7]:
class VerySimpleNet(nn.Module):
    def __init__(self, hidden_size_1=100, hidden_size_2=100):
        super(VerySimpleNet, self).__init__()
        self.quant = torch.quantization.QuantStub() # observer attached
        self.linear1 = nn.Linear(28*28, hidden_size_1)
        self.linear2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.linear3 = nn.Linear(hidden_size_2, 10)
        self.relu = nn.ReLU()
        self.dequant = torch.quantization.DeQuantStub() # observer attached

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.quant(x)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.linear3(x)
        x = self.dequant(x)
        return x

net = VerySimpleNet()

# Train the model

In [8]:
def train(train_loader, net, epochs=5, total_iterations_limit=None):
    cross_el = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

    total_iterations = 0

    for epoch in range(epochs):
        net.train()
        loss_sum = 0
        num_iterations = 0
        data_iterator = tqdm(train_loader, desc=f"Epoch {epoch+1}")
        if total_iterations_limit is not None:
            data_iterator.total = total_iterations_limit
        for data in data_iterator:
            num_iterations += 1
            total_iterations += 1
            x, y  =  data
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            output = net(x.view(-1, 28*28))
            loss = cross_el(output, y)
            loss_sum += loss.item()
            avg_loss = loss_sum / num_iterations
            data_iterator.set_postfix(loss=avg_loss)
            loss.backward()
            optimizer.step()

            if total_iterations_limit is not None and total_iterations >= total_iterations_limit:
                return

def print_size_of_model(model):
    torch.save(model.state_dict(), "temp_delme.p")
    print("Size (KB):", os.path.getsize("temp_delme.p"))
    os.remove("temp_delme.p")

def get_num_params(model):
    return sum(p.numel() for p in model.parameters())

# MODEL_FILENAME = "simplenet_qat.pt"

# if Path(MODEL_FILENAME).exists():
#     net.load_state_dict(torch.load(MODEL_FILENAME))
#     print("Model Loaded")
# else:
#     train(train_loader, net, epochs=5)
#     torch.save(net.state_dict(), MODEL_FILENAME)



Epoch 5: 100%|██████████| 6000/6000 [00:15<00:00, 389.84it/s, loss=0.0647]


# Insert min-max observers in the model

In [10]:
net.qconfig = torch.ao.quantization.default_qconfig
net.train()
net_quantized = torch.quantization.prepare_qat(net)
net_quantized


VerySimpleNet(
  (quant): QuantStub(
    (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
  )
  (linear1): Linear(
    in_features=784, out_features=100, bias=True
    (weight_fake_quant): MinMaxObserver(min_val=inf, max_val=-inf)
    (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
  )
  (linear2): Linear(
    in_features=100, out_features=100, bias=True
    (weight_fake_quant): MinMaxObserver(min_val=inf, max_val=-inf)
    (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
  )
  (linear3): Linear(
    in_features=100, out_features=10, bias=True
    (weight_fake_quant): MinMaxObserver(min_val=inf, max_val=-inf)
    (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
  )
  (relu): ReLU()
  (dequant): DeQuantStub()
)

# Train the model after adding the observers 

In [11]:
train(train_loader, net_quantized, epochs=5)

Epoch 5: 100%|██████████| 6000/6000 [00:17<00:00, 350.95it/s, loss=0.0447]


# Define the testing loop


In [19]:
def test(model: nn.Module, total_iterations: int = None):
    correct = 0
    total = 0
    iterations = 0
    model.eval()

    with torch.no_grad():
        for data in tqdm(test_loader, desc="Testing"):
            x, y = data
            x = x.to(device)
            y = y.to(device)
            output = model(x.view(-1, 28*28))
            for idx, i in enumerate(output):
                if torch.argmax(i) == y[idx]:
                    correct += 1
                total += 1
            iterations += 1
            if total_iterations is not None and iterations >= total_iterations:
                break

    print("Accuracy: ", {round(correct/total, 3)})
    

In [12]:
print(f"Check statistics of various layers")
net_quantized

Check statistics of various layers


VerySimpleNet(
  (quant): QuantStub(
    (activation_post_process): MinMaxObserver(min_val=-0.4242129623889923, max_val=2.821486711502075)
  )
  (linear1): Linear(
    in_features=784, out_features=100, bias=True
    (weight_fake_quant): MinMaxObserver(min_val=-1.7619431018829346, max_val=1.0522791147232056)
    (activation_post_process): MinMaxObserver(min_val=-122.23516082763672, max_val=77.81725311279297)
  )
  (linear2): Linear(
    in_features=100, out_features=100, bias=True
    (weight_fake_quant): MinMaxObserver(min_val=-1.1885151863098145, max_val=0.808742105960846)
    (activation_post_process): MinMaxObserver(min_val=-113.60478210449219, max_val=85.240966796875)
  )
  (linear3): Linear(
    in_features=100, out_features=10, bias=True
    (weight_fake_quant): MinMaxObserver(min_val=-1.228057622909546, max_val=0.3255835771560669)
    (activation_post_process): MinMaxObserver(min_val=-170.0066375732422, max_val=47.58976364135742)
  )
  (relu): ReLU()
  (dequant): DeQuantStub()


# Quantize the model using the statistics collected

In [13]:
torch.backends.quantized.engine = 'qnnpack' # this is needed else it throws an error
net_quantized.eval()
net_quantized = torch.ao.quantization.convert(net_quantized)


In [14]:
print(f'Check the statistics of the various layers after conversion')
net_quantized

Check the statistics of the various layers after conversion


VerySimpleNet(
  (quant): Quantize(scale=tensor([0.0256]), zero_point=tensor([17]), dtype=torch.quint8)
  (linear1): QuantizedLinear(in_features=784, out_features=100, scale=1.5752158164978027, zero_point=78, qscheme=torch.per_tensor_affine)
  (linear2): QuantizedLinear(in_features=100, out_features=100, scale=1.5657145977020264, zero_point=73, qscheme=torch.per_tensor_affine)
  (linear3): QuantizedLinear(in_features=100, out_features=10, scale=1.7133575677871704, zero_point=99, qscheme=torch.per_tensor_affine)
  (relu): ReLU()
  (dequant): DeQuantize()
)

# Print the weights of the model after quantization

In [15]:
# print the weights of the model after quantization
print('Weights after quantization')
print(torch.int_repr(net_quantized.linear1.weight()))

Weights after quantization
tensor([[5, 6, 3,  ..., 6, 5, 5],
        [1, 2, 2,  ..., 1, 2, 0],
        [6, 8, 5,  ..., 6, 7, 8],
        ...,
        [5, 5, 3,  ..., 3, 4, 1],
        [3, 4, 6,  ..., 4, 4, 4],
        [5, 5, 4,  ..., 6, 3, 4]], dtype=torch.int8)


# Compare the dequantized weights and the original weights

In [16]:
print('Original weights')
print(net.linear1.weight)
print('')
print("Quantized weights")
print(net_quantized.linear1.weight())

Original weights
Parameter containing:
tensor([[ 0.0392,  0.0586,  0.0101,  ...,  0.0614,  0.0432,  0.0415],
        [-0.0046,  0.0002,  0.0048,  ..., -0.0051,  0.0093, -0.0147],
        [ 0.0741,  0.1092,  0.0610,  ...,  0.0739,  0.0955,  0.1023],
        ...,
        [ 0.0537,  0.0574,  0.0228,  ...,  0.0174,  0.0367, -0.0003],
        [ 0.0605,  0.0686,  0.1007,  ...,  0.0810,  0.0810,  0.0805],
        [ 0.0415,  0.0362,  0.0220,  ...,  0.0584,  0.0091,  0.0293]],
       requires_grad=True)

Quantized weights
tensor([[0.0691, 0.0829, 0.0415,  ..., 0.0829, 0.0691, 0.0691],
        [0.0138, 0.0276, 0.0276,  ..., 0.0138, 0.0276, 0.0000],
        [0.0829, 0.1106, 0.0691,  ..., 0.0829, 0.0967, 0.1106],
        ...,
        [0.0691, 0.0691, 0.0415,  ..., 0.0415, 0.0553, 0.0138],
        [0.0415, 0.0553, 0.0829,  ..., 0.0553, 0.0553, 0.0553],
        [0.0691, 0.0691, 0.0553,  ..., 0.0829, 0.0415, 0.0553]],
       size=(100, 784), dtype=torch.qint8,
       quantization_scheme=torch.per_ten

# Print size and accuracy of the quantized model

In [17]:
print("Size of the model after quantization")
print_size_of_model(net_quantized)

Size of the model after quantization
Size (KB): 95266


In [20]:
print("Testing the model after quantization")
test(net_quantized)

Testing the model after quantization


Testing: 100%|██████████| 1000/1000 [00:01<00:00, 776.91it/s]

Accuracy:  {0.975}
